<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/Image_inpaint_New_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import gc
from tensorflow.keras.optimizers import Adam
import pandas as pd
import seaborn as sns

# Enable memory growth for GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Configuration
IMG_SIZE = 128  # Reduced from 256 due to memory constraints
BATCH_SIZE = 4
BUFFER_SIZE = 1000
DATA_DIR = "celeba_hq_256"

def load_and_analyze_dataset():
    """Load images and perform EDA"""
    image_paths = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith(('.jpg', '.png'))]
    print(f"Total images found: {len(image_paths)}")

    # Sample images for analysis
    sample_size = min(1000, len(image_paths))
    sample_paths = random.sample(image_paths, sample_size)

    # Analyze image properties
    sizes = []
    channels = []
    pixel_values = []

    for path in tqdm(sample_paths[:100]):  # Analyze first 100 images
        img = cv2.imread(path)
        sizes.append(img.shape[:2])
        channels.append(img.shape[2])
        pixel_values.extend(img.ravel())

    # Create visualizations
    plt.figure(figsize=(15, 5))

    plt.subplot(131)
    plt.hist(pixel_values, bins=50)
    plt.title('Pixel Value Distribution')

    plt.subplot(132)
    sizes_np = np.array(sizes)
    plt.scatter(sizes_np[:, 0], sizes_np[:, 1])
    plt.title('Image Dimensions')

    plt.subplot(133)
    plt.hist(channels)
    plt.title('Number of Channels')
    plt.show()

    return image_paths

def create_mask(height, width):
    """Create random masks for inpainting"""
    mask = np.ones((height, width, 1))

    # Random rectangular mask
    y1, x1 = np.random.randint(0, height - height//3), np.random.randint(0, width - width//3)
    h, w = height//3, width//3
    mask[y1:y1+h, x1:x1+w] = 0

    return mask

def preprocess_image(image_path):
    """Load and preprocess images"""
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32) / 255.0

    mask = create_mask(IMG_SIZE, IMG_SIZE)
    masked_img = img * mask

    return masked_img, mask, img

def create_dataset(image_paths):
    """Create TensorFlow dataset"""
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
    return dataset

def unet_model():
    """Create U-Net model"""
    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bridge
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.concatenate([conv2, up1], axis=-1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(up1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    up2 = layers.concatenate([conv1, up2], axis=-1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(up2)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(3, 1, activation='sigmoid')(conv5)

    return Model(inputs=inputs, outputs=outputs)

def hint_model():
    """Create HINT model with transformer architecture"""
    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))
    mask_input = layers.Input((IMG_SIZE, IMG_SIZE, 1))

    # Encode input with mask awareness
    x = layers.concatenate([inputs, mask_input])
    x = layers.Conv2D(64, 3, padding='same')(x)

    # Transformer blocks
    for _ in range(3):
        # Multi-head attention
        attention = layers.MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
        x = layers.Add()([attention, x])
        x = layers.LayerNormalization()(x)

        # FFN
        ffn = layers.Dense(128, activation='relu')(x)
        ffn = layers.Dense(64)(ffn)
        x = layers.Add()([ffn, x])
        x = layers.LayerNormalization()(x)

    outputs = layers.Conv2D(3, 1, activation='sigmoid')(x)

    return Model(inputs=[inputs, mask_input], outputs=outputs)

def combined_model(unet, hint):
    """Combine U-Net and HINT models"""
    inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))
    mask_input = layers.Input((IMG_SIZE, IMG_SIZE, 1))

    unet_output = unet(inputs)
    hint_output = hint([inputs, mask_input])

    combined = layers.Average()([unet_output, hint_output])

    return Model(inputs=[inputs, mask_input], outputs=combined)

def evaluate_models(models, test_dataset):
    """Evaluate models using various metrics"""
    metrics = {
        'PSNR': [],
        'SSIM': [],
        'L1_Loss': []
    }

    for model_name, model in models.items():
        psnr_values = []
        ssim_values = []
        l1_values = []

        for masked_imgs, masks, original_imgs in test_dataset:
            if isinstance(model.input, list):
                predictions = model([masked_imgs, masks])
            else:
                predictions = model(masked_imgs)

            # Calculate metrics
            psnr = tf.image.psnr(original_imgs, predictions, max_val=1.0)
            ssim = tf.image.ssim(original_imgs, predictions, max_val=1.0)
            l1 = tf.reduce_mean(tf.abs(original_imgs - predictions))

            psnr_values.extend(psnr.numpy())
            ssim_values.extend(ssim.numpy())
            l1_values.append(l1.numpy())

        metrics['PSNR'].append(np.mean(psnr_values))
        metrics['SSIM'].append(np.mean(ssim_values))
        metrics['L1_Loss'].append(np.mean(l1_values))

    # Visualize metrics
    df = pd.DataFrame(metrics, index=list(models.keys()))
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df)
    plt.title('Model Comparison')
    plt.xticks(rotation=45)
    plt.show()

    return df

def main():
    # 1. Load dataset and perform EDA
    image_paths = load_and_analyze_dataset()

    # 2. Split dataset
    train_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)

    # 3. Create datasets
    train_dataset = create_dataset(train_paths)
    test_dataset = create_dataset(test_paths)

    # 4. Create models
    unet = unet_model()
    hint = hint_model()
    combined = combined_model(unet, hint)

    # 5. Compile models
    optimizer = Adam(learning_rate=0.001)
    loss = 'mse'

    unet.compile(optimizer=optimizer, loss=loss)
    hint.compile(optimizer=optimizer, loss=loss)
    combined.compile(optimizer=optimizer, loss=loss)

    # 6. Train models (showing only structure here due to computational constraints)
    print("Models created and compiled successfully")

    # 7. Evaluate models
    models = {
        'U-Net': unet,
        'HINT': hint,
        'Combined': combined
    }

    results = evaluate_models(models, test_dataset)
    print("\nEvaluation Results:")
    print(results)

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'celeba_hq_256'